# Importing Libraries

In [1]:
from keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
from keras.utils import np_utils
import time

# Importing and Preprocessing Image Dataset

In [2]:
data_path = 'dataset'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories,labels))

img_size = 100
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path,category)
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        target.append(label_dict[category])

            
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target = np_utils.to_categorical(np.array(target))

# Instantiating CNN 

In [3]:
model=Sequential()

# Convolutional layer 1
model.add(Conv2D(256,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 2
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 3
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))

# FC layer 1
model.add(Dense(64,activation='relu'))

# FC layer 2
model.add(Dense(2,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 256)       2560      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       295040    
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 128)       1

# Training CNN network

In [4]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
train_data,test_data,train_target,test_target = train_test_split(data,target,train_size=0.9)
model.fit(train_data,train_target,epochs=20,validation_split=0.2)

Epoch 1/20
31/31 [==============================] - 53s 2s/step - loss: 0.6952 - accuracy: 0.5495 - val_loss: 0.6318 - val_accuracy: 0.6048
Epoch 2/20
31/31 [==============================] - 52s 2s/step - loss: 0.5615 - accuracy: 0.6929 - val_loss: 0.4561 - val_accuracy: 0.7863
Epoch 3/20
31/31 [==============================] - 52s 2s/step - loss: 0.3127 - accuracy: 0.8636 - val_loss: 0.6911 - val_accuracy: 0.6855
Epoch 4/20
31/31 [==============================] - 57s 2s/step - loss: 0.2721 - accuracy: 0.9051 - val_loss: 0.1266 - val_accuracy: 0.9476
Epoch 5/20
31/31 [==============================] - 51s 2s/step - loss: 0.1218 - accuracy: 0.9545 - val_loss: 0.1204 - val_accuracy: 0.9597
Epoch 6/20
31/31 [==============================] - 49s 2s/step - loss: 0.0817 - accuracy: 0.9717 - val_loss: 0.1395 - val_accuracy: 0.9355
Epoch 7/20
31/31 [==============================] - 49s 2s/step - loss: 0.0616 - accuracy: 0.9778 - val_loss: 0.1170 - val_accuracy: 0.9556
Epoch 8/20
31/31 [==

# Testing CNN, Haarcascade and Yolo on live webcam

In [5]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
yolo_net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
layer_names = yolo_net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

print_dict={0:'Allowed',1:'Not Allowed'}
color_dict={0:(0,255,0),1:(0,0,255)}

font = cv2.FONT_HERSHEY_SIMPLEX

In [6]:
source = cv2.VideoCapture(0)

starting_time= time.time()
frame_id = 0

while(True):
    _,frame= source.read() 
    frame_id+=1
    img = frame
    flag = 0
    
    height,width,channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) 

    yolo_net.setInput(blob)
    outs = yolo_net.forward(outputlayers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence)) 
                class_ids.append(class_id) 

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

    for i in indexes:
        i = i[0]
        box = boxes[i]
        if(class_ids[i] == 0):
            flag = 1
            cv2.rectangle(frame, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 0), 2)
            cv2.putText(frame,'person', (round(box[0])-10,round(box[1])-10),font, 0.5, (0, 0, 0), 2)

    if(flag == 1):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.1,4)  

        for (x,y,w,h) in faces:
            face_img = gray[y:y+w,x:x+w]
            resized = cv2.resize(face_img,(100,100))
            normalized = resized/255.0
            reshaped = np.reshape(normalized,(1,100,100,1))
            result = model.predict(reshaped)

            label = np.argmax(result,axis=1)[0]

            cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.putText(img, print_dict[label],(x, y-10),font,0.5,(255,255,255),2)


    elapsed_time = time.time() - starting_time
    fps = frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),1)
         
    cv2.imshow('LIVE',img)

    key = cv2.waitKey(1)           # press escape key to exit webcam
    if(key == 27):
        break
        
cv2.destroyAllWindows()
source.release()